使用許諾条件

①	設計一次エネルギー消費量算定法に基づく計算方法をPythonによりコーディングし、Jupyter notebook形式で保存したファイル（以下、本ファイルと呼ぶ）の著作権は、一般社団法人日本サステナブル建築協会に帰属します。
Copyright©2019. 一般社団法人日本サステナブル建築協会
All rights reserved. 

②	本ファイルは、住戸の設計一次エネルギー消費量算定方法に係る「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）2.1算定方法（第四章第七節 温水暖房）」に示す計算方法に基づき、国土交通省国土技術政策総合研究所・国立研究開発法人建築研究所、株式会社ユーワークス、株式会社住環境計画研究所が作成したものです。

③	本ファイルは、計算方法に示す計算過程の確認及び調査研究等に活用することを目的として作成されたものであり、一般社団法人日本サステナブル建築協会は著作権について放棄はしませんが、使用する各人の自己責任において仕様を変更することを妨げるものではありません。ただし、本ファイルによる計算結果を住宅の省エネルギー基準で規定する設計一次エネルギー消費量の結果として公的な申請に用いることはできません。

④	本ファイルを変更するかしないかを問わず、以下の条件を満たす場合に限り、頒布および使用が許可されます。
1.	頒布物の付属ドキュメント等の資料に、上記の著作権表示、本条件一覧、および下記免責条項を含めること。
2.	書面による特別の許可なしに、本ファイルから派生した製品の宣伝または販売の促進に、一般社団法人日本サステナブル建築協会、国土技術政策総合研究所・国立研究開発法人建築研究所および株式会社ユーワークス、株式会社住環境計画研究所の名前またはコントリビューターの名前を使用してはならない。

⑤	本ファイルは、著作権者およびコントリビューターによって「現状のまま」提供されており、明示黙示を問わず、商業的な使用可能性、および特定の目的に対する適合性に関する暗黙の保証も含め、またそれに限定されない、いかなる保証もありません。著作権もコントリビューターも、事由のいかんを問わず、損害発生の原因いかんを問わず、かつ責任の根拠が契約であるか厳格責任であるか（過失その他の）不法行為であるかを問わず、仮にそのような損害が発生する可能性を知らされていたとしても、本ファイルの使用によって発生した（代用品または代用サービスの調達、使用の喪失、データの喪失、利益の喪失、業務の中断も含め、またそれに限定されない）直接損害、間接損害、偶発的な損害、特別損害、懲罰的損害、または結果損害について、一切の責任を負わないものとします。



In [ ]:
import nbimporter
import numpy as np
from s04_07_common import get_Q_out_H_hs_d_t

## 付録E　電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機（給湯熱源：ガス瞬間式、暖房熱源：電気ヒートポンプ・ガス瞬間式併用） / Appendix E Electric heat pump, gas moment type combined hot water supply hot water heater (hot water supply heat source: gas moment type, heating heat source: electric heat pump, gas moment type combined use)

### E.1 記号及び単位 / Symbol and unit

省略

ommision

### E.2 エネルギー消費量 / Energy consumption

#### E.2.1 消費電力量 / Power consumption

1 時間当たりの熱源機の消費電力量$E_{E,hs}$は、1 時間当たりの熱源機の暖房出力$Q_{out,hs}$が0の場合は0とする。

The power consumption $ E_{E, hs} $ of the heat source unit per hour is 0 when the heating output $ Q_{out, hs} $ of the heat source unit per hour is 0.

$$\large
E_{E,hs} = E_{E,HPU} + E_{E,GU,aux} + E_{E,TU,aux}
$$

$E_{E,hs}$：1 時間当たりの熱源機の消費電力量(kWh/h) / Power consumption of heat source unit per hour  
$E_{E,HPU}$：1 時間当たりのヒートポンプユニットの消費電力量(kWh/h) / Power consumption of heat pump unit per hour  
$E_{E,GU,aux}$：1 時間当たりのガスユニットの補機の消費電力量(kWh/h) / Power consumption of gas unit accessories per hour  
$E_{E,TU,aux}$：1 時間当たりのタンクユニットの補機の消費電力量(kWh/h) / Power consumption of tank unit accessories per hour  

In [ ]:
def calc_E_E_hs(Q_dmd_H_hs_d_t, Theta_ex, Theta_RW_hs, h_ex, Theta_SW_d_t, TU_place):
    """ 1時間当たりの熱源機の消費電力量 (1)

    :param Q_dmd_H_hs_d_t: 1時間当たりの熱源機の熱需要 (MJ/h)
    :type Q_dmd_H_hs_d_t: ndarray
    :param Theta_ex: 外気温度
    :type Theta_ex: ndarray float
    :param Theta_RW_hs: 1時間平均の熱源機の戻り温水温度 (℃)
    :type Theta_RW_hs: ndarray
    :param h_ex: 外気相対湿度
    :type h_ex: ndarray
    :param Theta_SW_d_t: 往き温水温度 (℃)
    :type Theta_SW_d_t: ndarray
    :param TU_place: タンクユニットの設置場所
    :type TU_place: str
    :return: 1時間当たりの熱源機の消費電力量
    :rtype: ndarray
    """
    # ---------- タンクユニット周囲の空気温度 ----------

    # 1時間平均のタンクユニットの周囲空気温度 (℃) (23)
    Theta_TU_amb = calc_Theta_TU_amb(Theta_ex, TU_place)

    # ---------- 熱源機の平均暖房出力及び熱源機最大暖房出力 ----------

    # ガスユニットの定格暖房能力 (W)
    q_GU_rtd = get_q_GU_rtd()

    # 1時間当たりの熱源機の最大暖房出力 (MJ/h) (22)
    Q_max_hs = get_Q_max_hs(q_GU_rtd)

    # 1時間当たりの温水暖房用熱源機の暖房出力 (MJ/h)
    Q_out_hs = get_Q_out_H_hs_d_t(Q_dmd_H_hs_d_t, Q_max_hs)

    # 1時間平均の熱源機の暖房出力 (W) (21)
    q_out_hs = get_q_out_hs(Q_out_hs)

    # ----------- 暖房出力分担率 -----------

    # ヒートポンプ最大運転時のヒートポンプユニット運転時のガスユニットの熱損失 (W) (20)
    q_dash_GU_loss_HPU = calc_q_dash_GU_loss_HPU(Theta_SW_d_t, Theta_TU_amb)

    # ヒートポンプ最大運転時のタンクユニットのタンクの熱損失 (W) (19)
    q_dash_TU_tank_loss = calc_q_dash_TU_tank_loss(Theta_TU_amb)

    # タンクユニットのヒートポンプ配管の熱損失 (W) (18)
    q_dash_TU_pipe_loss = calc_q_dash_TU_pipe_loss(Theta_SW_d_t, Theta_ex)

    # ヒートポンプユニットの最大暖房出力 (W) (17)
    q_HPU_max = calc_q_HPU_max(Theta_ex, Theta_RW_hs)

    # ヒートポンプユニットの暖房出力分担率 (-) (16)
    r_HPU = get_r_HPU(q_HPU_max, q_dash_TU_tank_loss, q_dash_TU_pipe_loss, q_out_hs, q_dash_GU_loss_HPU)

    # ガスユニットの暖房出力分担率 (-) (15)
    r_GU = get_r_GU(r_HPU)

    # ---------- ガスユニットのガス消費量 ----------

    # 1時間平均のガスユニットのガス消費量 (W) (14b)
    q_G_GU = calc_q_G_GU(q_out_hs, Theta_SW_d_t)

    # ---------- ガスユニットの補機の消費電力 ----------

    # 1時間当たりのガスユニットの補機の消費電力 (kWh/h) (13)
    E_E_GU_aux = calc_E_E_GU_aux(q_G_GU, r_HPU, r_GU)

    # ---------- ヒートポンプユニットの暖房出力 ----------

    # 熱源機のヒートポンプユニット分断暖房出力 (MJ/h) (12)
    Q_out_hs_HPU = get_Q_out_hs_HPU(Q_out_hs, r_HPU)

    # ヒートポンプユニット運転時のガスユニットの熱損失 (MJ/h) (11)
    Q_GU_loss_HPU = calc_Q_GU_loss_HPU(Theta_SW_d_t, Theta_TU_amb, r_HPU)

    # 1時間平均のヒートポンプユニットの出湯温度 (℃)
    Theta_HPU_out = get_Theta_HPU_out(r_HPU, Theta_SW_d_t)

    # タンクユニットのタンクの熱損失 (MJ/h) (10)
    Q_TU_tank_loss = calc_Q_TU_tank_loss(Theta_HPU_out, Theta_TU_amb)

    # タンクユニットのヒートポンプ配管の熱損失 (MJ/h) (9)
    Q_TU_pipe_loss = calc_Q_TU_pipe_loss(Theta_SW_d_t, Theta_ex)

    # ヒートポンプユニットの暖房出力 (MJ/h) (8)
    Q_HPU = get_Q_HPU(Q_out_hs_HPU, Q_GU_loss_HPU, Q_TU_tank_loss, Q_TU_pipe_loss)

    # ---------- ヒートポンプユニットの消費電力量 ----------

    # ヒートポンプユニットの平均暖房出力,最小暖房出力 (6)(7)
    q_HPU_ave = get_q_HPU_ave(Q_HPU)
    q_HPU_min = get_q_HPU_min(Theta_ex, Theta_RW_hs, Theta_HPU_out)

    # 1時間平均のヒートポンプユニットの断続運転率 (-) (5)
    r_intmit = get_r_intmit(q_HPU_ave, q_HPU_min)

    # 1時間当たりのヒートポンプユニットの消費電力量 (kWh/h) (4)
    E_E_HPU = calc_E_E_HPU(q_HPU_ave, q_HPU_min, Theta_ex, h_ex, Theta_RW_hs, Theta_HPU_out)

    # ---------- タンクユニットの補機消費電力量 ----------

    # 1時間当たりのタンクユニットの補機の消費電力量 (kWh/h) (3)
    E_E_TU_aux = calc_E_E_TU_aux(r_intmit)

    # ---------- 熱源機の消費電力量 ----------

    # 1時間当たりの熱源機の消費電力量 (kWh/h) (1)
    E_E_hs = E_E_HPU + E_E_GU_aux + E_E_TU_aux
    E_E_hs[Q_out_hs == 0] = 0

    return E_E_hs

#### E.2.2 ガス消費量 / Gas consumption

1 時間当たりの熱源機のガス消費量$E_{G,hs}$は、1 時間当たりの熱源機の暖房出力$Q_{out,hs}$が0の場合は0とする。

The gas consumption $ E_ {G, hs} $ of the heat source unit per hour is 0 when the heating output $ Q_ {out, hs} $ of the heat source unit per hour is 0.

$$\large
E_{G,hs} = E_{G,GU}
$$

$E_{G,hs}$：1 時間当たりの熱源機のガス消費量(MJ/h) / Gas consumption of heat source unit per hour  
$E_{G,GU}$：1 時間当たりのガスユニットのガス消費量(MJ/h) / Gas unit gas consumption per hour  

In [ ]:
def calc_E_G_hs(Theta_ex, Theta_SW_d_t, Theta_RW_hs, TU_place, Q_dmd_H_hs_d_t):
    """ 熱源機のガス消費量 (2)

    :param Theta_ex: 外気温度（℃）
    :type Theta_ex: ndarray
    :param Theta_SW_d_t: 往き温水温度 (℃)
    :type Theta_SW_d_t: ndarray
    :param Theta_RW_hs: 1時間平均の熱源機の戻り温水温度 (℃)
    :type Theta_RW_hs: ndarray
    :param TU_place: タンクユニットの設置場所
    :type TU_place: str
    :param Q_dmd_H_hs_d_t: 1時間当たりの熱源機の熱需要 (MJ/h)
    :type Q_dmd_H_hs_d_t: ndarray
    :return: 熱源機のガス消費量
    :rtype: ndarray
    """
    # タンクユニットの周囲空気温度 (℃) (23)
    Theta_TU_amb = calc_Theta_TU_amb(Theta_ex, TU_place)

    # ガスユニットの定格暖房能力 (W)
    q_GU_rtd = get_q_GU_rtd()

    # 熱源機の最大暖房出力 (22)
    Q_max_hs = get_Q_max_hs(q_GU_rtd)

    # 温水暖房用熱源機の暖房出力
    Q_out_hs = get_Q_out_H_hs_d_t(Q_dmd_H_hs_d_t, Q_max_hs)

    # 熱源機の暖房出力 (21)
    q_out_hs = get_q_out_hs(Q_out_hs)

    # ヒートポンプ最大運転時のヒートポンプユニット運転時のガスユニットの熱損失 (20)
    q_dash_GU_loss_HPU = calc_q_dash_GU_loss_HPU(Theta_SW_d_t, Theta_TU_amb)

    # ヒートポンプ最大運転時のタンクユニットのタンクの熱損失 (19)
    q_dash_TU_tank_loss = calc_q_dash_TU_tank_loss(Theta_TU_amb)

    # タンクユニットのヒートポンプ配管の熱損失 (18)
    q_dash_TU_pipe_loss = calc_q_dash_TU_pipe_loss(Theta_SW_d_t, Theta_ex)

    # ヒートポンプユニットの最大暖房出力 (17)
    q_HPU_max = calc_q_HPU_max(Theta_ex, Theta_RW_hs)

    # ヒートポンプユニットの暖房出力分担率 (16)
    r_HPU = get_r_HPU(q_HPU_max, q_dash_TU_tank_loss, q_dash_TU_pipe_loss, q_out_hs, q_dash_GU_loss_HPU)

    # ガスユニットの暖房出力分担率 (15)
    r_GU = get_r_GU(r_HPU)

    # ガスユニットのガス消費量 (14a)
    E_G_hs = calc_E_G_GU(r_GU, q_out_hs, Theta_SW_d_t)
    E_G_hs[Q_out_hs == 0] = 0

    return E_G_hs

#### E.2.3 灯油消費量 / Kerosene consumption

1 時間当たりの熱源機の灯油消費量$E_{K,hs}$は0とする。 

The kerosene consumption $ E_ {K, hs} $ of the heat source machine per hour is 0.

In [ ]:
def get_E_K_hs():
    """ 熱源機の灯油消費量

    :return: 熱源機の灯油消費量
    :rtype: ndarray
    """
    return np.zeros(24 * 365)


#### E.2.4 その他の燃料の一次エネルギー消費量 / Primary energy consumption of other fuels

1 時間当たりの熱源機のその他の燃料の一次エネルギー消費量$E_{M,hs}$は0とする。 

Primary energy consumption $ E_{M, hs} $ of the other fuels of the heat source unit per hour is assumed to be zero.

In [ ]:
def get_E_M_hs():
    """ 熱源機のその他の燃料の一次エネルギー消費量

    :return: 熱源機のその他の燃料の一次エネルギー消費量
    :rtype: ndarray
    """
    return np.zeros(24 * 365)


### E.3 消費電力量及びガス消費量 / Power consumption and gas consumption

#### E.3.1 タンクユニットの補機消費電力量 / Auxiliary power consumption of tank unit

$$\large
E_{E,TU,aux} = P_{TU,aux} \times 10^{-3}
$$

$$\large
P_{TU,aux} = f_{TU,P,aux} (r_{intmit})
$$

$P_{TU,aux}$：1 時間平均のタンクユニットの補機の消費電力(W) / Power consumption of 1 hour average tank unit accessories  
$r_{intmit}$：1 時間平均のヒートポンプユニットの断続運転率 / Intermittent operation rate of heat pump unit with one hour average

In [ ]:
def calc_E_E_TU_aux(r_intmit):
    """ 1時間当たりのタンクユニットの補機消費電力量 (3)

    :param r_intmit: 1時間平均のヒートポンプユニットの断続運転率
    :type r_intmit: ndarray
    :return: 1時間当たりのタンクユニットの補機消費電力量 (3)
    :rtype: ndarray
    """
    # 1時間平均のタンクユニットの補機の消費電力 (3b)
    P_TU_aux = f_TU_P_aux(r_intmit)

    # タンクユニットの補機消費電力量 (3a)
    E_E_TU_aux = P_TU_aux * 10 ** (-3)

    return E_E_TU_aux

#### E.3.2 ヒートポンプユニットの消費電力量 / Power consumption of heat pump unit

$q_{HPU,ave} \geq q_{HPU,min} $ の場合（連続運転の場合）

In the case of $q_{HPU,ave} \geq q_{HPU,min} $ (in the case o continuous operation)

$$\large
E_{E,HPU} = P_{HPU,ctn} \times 10^{-3}
$$

$$\large
P_{HPU,ctn} = f_{HPU,P} (\theta_{ex}, h_{ex}, \theta_{RW,hs}, \theta_{HPU,out}, q_{HPU,ave})
$$

$q_{HPU,ave} < q_{HPU,min} $ の場合（断続運転の場合）

In the case of $q_{HPU,ave} < q_{HPU,min} $ (in the case of intermittent operation)

$$\large
E_{E,HPU} = \frac{q_{HPU,ave}}{\frac{q_{HPU,min}}{P_{HPU,min}} \times (1 - CD \times (1 - r_{intmit}))} \times 10^{-3}
$$

$$\large
P_{HPU,min} = f_{HPU,P} (\theta_{ex}, h_{ex}, \theta_{RW,hs}, \theta_{HPU,out}, q_{HPU,min})
$$

$q_{HPU,ave}$：1 時間平均のヒートポンプユニットの暖房出力(W) / Heating output of heat pump unit of 1 hour average  
$q_{HPU,min}$：1 時間平均のヒートポンプユニットの最小暖房出力(W) / 1 hour average heat pump unit minimum heating output  
$P_{HPU,ctn}$：1 時間平均の連続運転時のヒートポンプユニットの消費電力(W) / Power consumption of the heat pump unit during one hour average continuous operation  
$P_{HPU,min}$：1 時間平均のヒートポンプユニットの最小消費電力(W) / 1 hour average heat pump unit minimum power consumption  
$\theta_{ex}$：1 時間平均の外気温度(℃) / 1 hour average outside temperature  
$h_{ex}$：1 時間平均の外気相対湿度(％) / 1 hour average ambient air relative humidity  
$\theta_{RW,hs}$：1 時間平均の熱源機の戻り温水温度(℃) / One hour average return hot water temperature of heat source machine  
$\theta_{HPU,out}$：1 時間平均のヒートポンプユニットの出湯温度(℃) / 1 hour average hot water outlet temperature of heat pump unit  
$CD$：CD 値 / CD value  
$r_{intmit}$：ヒートポンプユニットの 1 時間平均の断続運転率 / One hour average intermittent operation rate of heat pump unit

CD値は0.23とする。

The CD value is 0.23.

In [ ]:
def calc_E_E_HPU(q_HPU_ave, q_HPU_min, Theta_ex, h_ex, Theta_RW_hs, Theta_HPU_out):
    """ 1時間当たりのヒートポンプユニットの消費電力量 (4)

    :param q_HPU_ave: 1時間平均のヒートポンプユニットの暖房出力 (W)
    :type q_HPU_ave: ndarray
    :param q_HPU_min: 1時間平均のヒートポンプユニットの最小暖房出力 (W)
    :type q_HPU_min: ndarray
    :param Theta_ex: 1時間平均の外気温度 (℃)
    :type Theta_ex: ndarray
    :param h_ex: 1時間平均の外気相対湿度 (%)
    :type h_ex: ndarray
    :param Theta_RW_hs: 1時間平均の熱源機の戻り温水温度 (℃)
    :type Theta_RW_hs: ndarray
    :param Theta_HPU_out: 1時間平均のヒートポンプユニットの出湯温度 (℃)
    :type Theta_HPU_out: ndarray
    :return: 1時間当たりのヒートポンプユニットの消費電力量 (4)
    :rtype: ndarray
    """
    # E_E_HPU の作業領域を確保
    E_E_HPU = np.zeros(24 * 365)

    # ==== 連続運転の場合 ====
    f_ctn = q_HPU_ave >= q_HPU_min

    # 1時間平均の連続運転時のヒートポンプユニットの消費電力 (W) (4a-2)
    P_HPU_ctn = f_HPU_p(Theta_ex, h_ex, Theta_RW_hs, Theta_HPU_out, q_HPU_ave)

    # 1時間当たりのヒートポンプユニットの消費電力量 (kW) (4a-1)
    E_E_HPU[f_ctn] = P_HPU_ctn[f_ctn] * 10 ** (-3)

    # ==== 断続運転の場合 ====
    f_intmit = q_HPU_ave < q_HPU_min

    # 1時間平均のヒートポンプユニットの断続運転率 (5)
    r_intmit = get_r_intmit(q_HPU_ave, q_HPU_min)

    # 1時間平均のヒートポンプユニットの最小消費電力 (kW) (4b-2)
    P_HPU_min = f_HPU_p(Theta_ex, h_ex, Theta_RW_hs, Theta_HPU_out, q_HPU_min)

    # 1時間当たりのヒートポンプユニットの消費電力量 (kW) (4b-1)
    CD = 0.23
    E_E_HPU[f_intmit] = q_HPU_ave[f_intmit] / (
            q_HPU_min[f_intmit] / P_HPU_min[f_intmit] * (1 - CD * (1 - r_intmit[f_intmit]))) * 10 ** (-3)

    return E_E_HPU

1 時間平均のヒートポンプユニットの断続運転率$r_{intmit}$は、1.0を超えない。

The intermittent operation rate $ r_{intmit} $ of the 1 hour average heat pump unit does not exceed 1.0.

$$\large
r_{intmit} = \frac{q_{HPU,ave}}{q_{HPU,min}}
$$

$q_{HPU,ave}$：1 時間平均のヒートポンプユニットの暖房出力(W) / Heating output of heat pump unit of 1 hour average  
$q_{HPU,min}$：1 時間平均のヒートポンプユニットの最小暖房出力(W) / 1 hour average heat pump unit minimum heating output  

In [ ]:
def get_r_intmit(q_HPU_ave, q_HPU_min):
    """ 1時間平均のヒートポンプユニットの断続運転率 (5)

    :param q_HPU_ave: 1時間平均のヒートポンプユニットの平均暖房出力 (W)
    :type q_HPU_ave: ndarray
    :param q_HPU_min: 1時間平均のヒートポンプユニットの最小暖房出力 (W)
    :type q_HPU_min: ndarray
    :return: 1時間平均のヒートポンプユニットの断続運転率 (5)
    :rtype: ndarray
    """
    r_intmit = np.clip(q_HPU_ave / q_HPU_min, None, 1.0)
    return r_intmit

$$\large
q_{HPU,min} = f_{HPU,q,min} (\theta_{ex}, \theta_{RW,hs}, \theta_{HPU,out})
$$

$\theta_{ex}$：1 時間平均の外気温度(℃) / 1 hour average outside temperature  
$\theta_{RW,hs}$：1 時間平均の熱源機の戻り温水温度(℃) / One hour average return hot water temperature of heat source machine  
$\theta_{HPU,out}$：1 時間平均のヒートポンプユニットの出湯温度(℃) / 1 hour average hot water outlet temperature of heat pump unit  

In [ ]:
def get_q_HPU_min(Theta_ex, Theta_RW_hs, Theta_HPU_out):
    """ 1時間平均のヒートポンプユニットの最小暖房出力 (6)

    :param Theta_ex:  1時間平均の外気温度 (℃)
    :type Theta_ex: ndarray
    :param Theta_RW_hs: 1時間平均の熱源機の戻り温水温度 (℃)
    :type Theta_RW_hs: ndarray
    :param Theta_HPU_out: 1時間平均のヒートポンプユニットの出湯温度 (℃)
    :type Theta_HPU_out: ndarray
    :return: 1時間平均のヒートポンプユニットの最小暖房出力 (6)
    :rtype: ndarray
    """
    return f_HPU_q_min(Theta_ex, Theta_RW_hs, Theta_HPU_out)


$$\large
q_{HPU,ave} = Q_{HPU} \times 10^{6} \div 3600
$$

$Q_{HPU}$：1 時間当たりのヒートポンプユニットの暖房出力(MJ/h) / Heating output of heat pump unit per hour  

In [ ]:
def get_q_HPU_ave(Q_HPU):
    """ 1時間平均のヒートポンプユニットの平均暖房出力 (7)

    :param Q_HPU: 1時間当たりのヒートポンプユニットの暖房出力 (MJ/h)
    :type Q_HPU: ndarray
    :return: 1時間平均のヒートポンプユニットの平均暖房出力 (7)
    :rtype: ndarray
    """
    return Q_HPU * 10 ** 6 / 3600

#### E.3.3 ヒートポンプユニットの暖房出力 / Heating output of heat pump unit

$$\large
Q_{HPU} = Q_{out,hs,HPU} + Q_{GU,loss,HPU} + Q_{TU,tank,loss} + Q_{TU,pipe,loss}
$$

$Q_{out,hs,HPU}$：1 時間当たりの熱源機のヒートポンプユニット分担暖房出力(MJ/h) / Heat pump unit shared heating output of heat source unit per hour  
$Q_{GU,loss,HPU}$：1 時間当たりのヒートポンプユニット運転時のガスユニットの熱損失量(MJ/h) / Heat loss of gas unit during operation of heat pump unit per hour  
$Q_{TU,tank,loss}$：1 時間当たりのタンクユニットのタンクの熱損失量(MJ/h) / Heat loss of tank unit tank per hour  
$Q_{TU,pipe,loss}$：1 時間当たりのタンクユニットのヒートポンプ配管の熱損失量(MJ/h) / Heat loss of heat pump piping of tank unit per hour  

In [ ]:
def get_Q_HPU(Q_out_hs_HPU, Q_GU_loss_HPU, Q_TU_tank_loss, Q_TU_pipe_loss):
    """ 1時間当たりのヒートポンプユニットの暖房出力 (8)

    :param Q_out_hs_HPU: 1時間当たりの熱源機のヒートポンプユニット分担暖房出力 (MJ/h)
    :type Q_out_hs_HPU: ndarray
    :param Q_GU_loss_HPU: 1時間当たりのヒートポンプユニット運転時のガスユニットの熱損失量 (MJ/h)
    :type Q_GU_loss_HPU: ndarray
    :param Q_TU_tank_loss: 1時間当たりのタンクユニットのタンクの熱損失量 (MJ/h)
    :type Q_TU_tank_loss: ndarray
    :param Q_TU_pipe_loss: 1時間当たりのタンクユニットのヒートポンプ配管の熱損失量 (MJ/h)
    :type Q_TU_pipe_loss: ndarray
    :return: 1時間当たりのヒートポンプユニットの暖房出力
    :rtype: ndarray
    """
    Q_HPU = Q_out_hs_HPU + Q_GU_loss_HPU + Q_TU_tank_loss + Q_TU_pipe_loss
    return Q_HPU

$$\large
Q_{TU,pipe,loss} = q_{TU,pipe,loss} \times 3600 \times 10^{-6}
$$

$$\large
q_{TU,pipe,loss} = f_{TU,q,HPP} (\theta_{SW,hs}, \theta_{ex})
$$

$q_{TU,pipe,loss}$：1 時間平均のタンクユニットのヒートポンプ配管の熱損失(W) / Heat loss of heat pump piping of tank unit of 1 hour average  
$\theta_{SW,hs}$：1 時間平均の熱源機の往き温水温度(℃) / 1 hour average hot water temperature of the heat source machine  
$\theta_{ex}$：1 時間平均の外気温度(℃) / 1 hour average outside temperature  

In [ ]:
def calc_Q_TU_pipe_loss(Theta_SW_hs, Theta_ex):
    """ 1時間当たりのタンクユニットのヒートポンプ管の熱損失量 (9)

    :param Theta_SW_hs: 1時間平均の熱源機の往き温水温度 (℃)
    :type Theta_SW_hs: ndarray
    :param Theta_ex: 外気温度 (℃)
    :type Theta_ex: ndarray
    :return: 1時間当たりのタンクユニットのヒートポンプ管の熱損失量 (MJ/h)
    :rtype: ndarray
    """
    # タンクユニットのヒートポンプ配管の熱損失 (W) (9b)
    q_TU_pipe_loss = f_TU_q_HPP(Theta_SW_hs, Theta_ex)

    # タンクユニットのヒートポンプ管の熱損失量 (MJ/h) (9a)
    Q_TU_pipe_loss = q_TU_pipe_loss * 3600 * 10 ** (-6)

    return Q_TU_pipe_loss

$$\large
Q_{TU,tank,loss} = q_{TU,tank,loss} \times 3600 \times 10^{-6}
$$

$$\large
q_{TU,tank,loss} = f_{TU,q,tank} (\theta_{HPU,out}, \theta_{TU,amb})
$$

$q_{TU,tank,loss}$：1 時間平均のタンクユニットのタンクの熱損失(W) / 1 hour average tank unit tank heat loss  
$\theta_{HPU,out}$：1 時間平均のヒートポンプユニットの出湯温度(℃) / 1 hour average hot water outlet temperature of heat pump unit  
$\theta_{TU,amb}$：1 時間平均のタンクユニットの周囲空気温度(℃) / 1 hour average tank unit ambient air temperature  

In [ ]:
def calc_Q_TU_tank_loss(Theta_HPU_out, Theta_TU_amb):
    """ 1時間当たりのタンクユニットのタンクの熱損失量 (10)

    :param Theta_HPU_out: 1時間平均のヒートポンプユニットの出湯温度 (℃)
    :type Theta_HPU_out: ndarray
    :param Theta_TU_amb: 1時間平均のタンクユニットの周囲空気温度 (℃)
    :type Theta_TU_amb: ndarray
    :return: 1時間当たりのタンクユニットのタンクの熱損失量 (MJ/h)
    :rtype: ndarray
    """
    # タンクユニットのタンクの熱損失 (W) (10b)
    q_TU_tank_loss = f_TU_q_tank(Theta_HPU_out, Theta_TU_amb)

    # タンクユニットのタンクの熱損失量 (MJ/h) (10a)
    Q_TU_tank_loss = q_TU_tank_loss * 3600 * 10 ** (-6)

    return Q_TU_tank_loss

1 時間平均のヒートポンプユニットの出湯温度$\theta_{HPU,out}$は、1 時間平均の熱源機のヒートポンプユニットの暖房出力分担率$r_{HPU}$が1.0を下回る場合は85℃とし、1.0に等しいかつ 1 時間平均の熱源機の往き温水温度$\theta_{SW,hs}$が40℃の場合は65℃とし、1.0に等しいかつ 1 時間平均の熱源機の往き温水温度$\theta_{SW,hs}$が60℃の場合は85℃とする。 

The hot water temperature $\theta_{HPU,out}$ of the 1-hour average heat pump unit is 85 ° C. when the heating output sharing ratio $r_{HPU}$ of the 1-hour average heat source unit heat pump unit is less than 1.0, equal to 1.0 and 1-hour average hot water source heat source machine When the temperature $\theta_{SW,hs}$ is 40 ° C, the temperature is 65 ° C, and when it is equal to 1.0 and the 1 hour average hot-water heater temperature $\theta_{SW,hs}$ is 60 ° C, the temperature is 85 ° C.

In [ ]:
def get_Theta_HPU_out(r_HPU, Theta_SW_hs):
    """ 1時間平均のヒートポンプユニットの出湯温度

    :param r_HPU: 1時間平均のヒートポンプユニット暖房出力分担率 (-)
    :type r_HPU: ndarray
    :param Theta_SW_hs: 熱源機の往き温水温度 (℃)
    :type Theta_SW_hs: ndarray
    :return: 1時間平均のヒートポンプユニットの出湯温度 (℃)
    :rtype: ndarray
    """
    Theta_HPU_out = np.zeros(24 * 365)
    Theta_HPU_out[r_HPU < 1.0] = 85.0
    Theta_HPU_out[np.logical_and(r_HPU == 1.0, Theta_SW_hs == 40.0)] = 65.0
    Theta_HPU_out[np.logical_and(r_HPU == 1.0, Theta_SW_hs == 60.0)] = 85.0
    return Theta_HPU_out


$$\large
Q_{GU,loss,HPU} = q_{GU,loss,HPU} \times r_{HPU} \times 3600 \times 10^{-6}
$$

$$\large
q_{GU,loss,HPU} = f_{GU,q,lossHP} (\theta_{SW,hs}, \theta_{TU,amb})
$$

$q_{GU,loss,HPU}$：1 時間平均のヒートポンプユニット運転時のガスユニットの熱損失(W) / Heat loss of gas unit during operation of heat pump unit with one hour average  
$\theta_{SW,hs}$：1 時間平均の熱源機の往き温水温度(℃) / 1 hour average hot water temperature of the heat source machine  
$\theta_{TU,amb}$：1 時間平均のタンクユニットの周囲空気温度(℃) / 1 hour average tank unit ambient air temperature  
$r_{HPU}$：1 時間平均のヒートポンプユニットの暖房出力分担率 / 1 hour average heating output share rate of heat pump unit  

In [ ]:
def calc_Q_GU_loss_HPU(Theta_SW_hs, Theta_TU_amb, r_HPU):
    """ 1時間当たりのヒートポンプユニット運転時のガスユニットの熱損失量 (11)

    :param Theta_SW_hs: 温水暖房用熱源機の往き温水温度
    :type Theta_SW_hs: ndarray
    :param Theta_TU_amb: 1時間平均のタンクユニットの周囲空気温度 (℃)
    :type Theta_TU_amb: ndarray
    :param r_HPU: 1時間平均のヒートポンプユニット暖房出力分担率 (-)
    :type r_HPU: ndarray
    :return: 1時間当たりのヒートポンプユニット運転時のガスユニットの熱損失量 (MJ/h)
    :rtype: ndarray
    """
    # ヒートポンプユニット運転時のガスユニットの熱損失 (W) (11b)
    q_GU_loss_HPU = f_GU_q_lossHP(Theta_SW_hs, Theta_TU_amb)

    # ヒートポンプユニット運転時のガスユニットの熱損失量 (MJ/h) (11a)
    Q_GU_loss_HPU = q_GU_loss_HPU * r_HPU * 3600 * 10 ** (-6)

    return Q_GU_loss_HPU

$$\large
Q_{out,hs,HPU} = Q_{out,hs} \times r_{HPU}
$$

$Q_{out,hs}$：1 時間平均の熱源機の暖房出力(MJ/h) / 1 hour average heating output of heat source machine  
$r_{HPU}$：1 時間平均のヒートポンプユニットの暖房出力分担率 / 1 hour average heating output share rate of heat pump unit  

In [ ]:
def get_Q_out_hs_HPU(Q_out_hs, r_HPU):
    """ 1時間当たりの熱源機のヒートポンプユニット分断暖房出力 (12)

    :param Q_out_hs: 1時間平均の熱源機の暖房出力 (MJ/h)
    :type Q_out_hs: ndarray
    :param r_HPU: 1時間平均のヒートポンプユニット暖房出力分担率 (-)
    :type r_HPU: ndarray
    :return: 1時間当たりの熱源機のヒートポンプユニット分断暖房出力 (MJ/h)
    :rtype: ndarray
    """
    return Q_out_hs * r_HPU

#### E.3.4 ガスユニットの補機の消費電力 / Power consumption of gas unit accessories

$$\large
E_{E,GU,aux} = (P_{GU,aux,ON} \times r_{GU} + P_{GU,aux,OFF} \times r_{HPU}) \times 10^{-3}
$$

$$\large
P_{GU,aux,ON} = f_{GU,P,aux} (q_{G,GU})
$$

$$\large
P_{GU,aux,OFF} = f_{GU,P,aux} (0.0)
$$

$P_{GU,aux,ON}$：1 時間平均のガス燃焼時のガスユニットの補機の消費電力(W) / Power consumption of gas unit accessories during 1 hour average gas combustion  
$P_{GU,aux,OFF}$：1 時間平均のガス非燃焼時のガスユニットの補機の消費電力(W) / Power consumption of auxiliary unit of gas unit during non-gassing of one hour average gas  
$r_{GU}$：1 時間平均のガスユニットの暖房出力分担率 / 1 hour average gas unit heating output share rate  
$r_{HPU}$：1 時間平均のヒートポンプユニットの暖房出力分担率 / 1 hour average heating output share rate of heat pump unit  
$q_{G,GU}$：1 時間平均のガスユニットのガス消費量(W) / 1 hour average gas unit gas consumption  

In [ ]:
def calc_E_E_GU_aux(q_G_GU, r_HPU, r_GU):
    """ 1時間当たりのガスユニットの補機の消費電力 (13)

    :param q_G_GU: ガスユニットのガス消費量 (W)
    :type q_G_GU: ndarray
    :param r_HPU: 1時間平均のヒートポンプユニット暖房出力分担率 (-)
    :type r_HPU: ndarray
    :param r_GU: 1時間平均のガスユニットの暖房出力分担率 (-)
    :type r_GU: ndarray
    :return: 1時間当たりのガスユニットの補機の消費電力
    :rtype: ndarray
    """
    # 1時間平均のガス非燃焼時のガスユニットの補機の消費電力 (W) (13c)
    P_GU_aux_OFF = f_GU_P_aux(0.0)

    # 1時間平均のガス燃焼時のガスユニットの補機の消費電力 (W) (13b)
    P_GU_aux_ON = f_GU_P_aux(q_G_GU)

    # 1時間当たりのガスユニットの補機の消費電力 (13a)
    E_E_GU_aux = (P_GU_aux_ON * r_GU + P_GU_aux_OFF * r_HPU) * 10 ** (-3)

    return E_E_GU_aux

#### E.3.5 ガスユニットのガス消費量 / Gas unit gas consumption

$$\large
E_{E,GU} = q_{G,GU} \times r_{GU} \times 3600 \times 10^{-6}
$$

$$\large
q_{G,GU} = f_{GU,G} (q_{GU}, e_{GU,rtd}, q_{GU,rtd})
$$

$$\large
q_{GU} = min(q_{out,hs}, q_{GU,rtd})
$$

$q_{G,GU}$：1 時間平均のガスユニットのガス消費量(W) / 1 hour average gas unit gas consumption  
$r_{GU}$：1 時間平均のガスユニットの暖房出力分担率 / 1 hour average gas unit heating output share rate  
$q_{GU}$：1 時間平均のガスユニットの暖房出力(W) / 1 hour average gas unit heating output  
$q_{out,hs}$：1 時間平均の熱源機の暖房出力(W) / 1 hour average heating output of heat source machine  
$e_{GU,rtd}$：ガスユニットの定格暖房効率 / Rated heating efficiency of gas unit  
$q_{GU,rtd}$：ガスユニットの定格暖房能力(W) / Gas unit rated heating capacity  

In [ ]:
def calc_E_G_GU(r_GU, q_out_hs, Theta_SW_hs):
    """ 1時間平均のガスユニットのガス消費量 (MJ/h) (14a)

    :param r_GU: 1時間平均のガスユニットの暖房出力分担率 (-)
    :type r_GU: ndarray
    :param q_out_hs: 1時間平均の熱源機の暖房出力 (W)
    :type q_out_hs: ndarray
    :param Theta_SW_hs: 温水暖房用熱源機の往き温水温度
    :type Theta_SW_hs: ndarray
    :return: 1時間平均のガスユニットのガス消費量 (MJ/h)
    :rtype: ndarray
    """
    # 1時間平均のガスユニットのガス消費量 (14b)
    q_G_GU = calc_q_G_GU(q_out_hs, Theta_SW_hs)

    # ガスユニットのガス消費量 (14a)
    E_G_GU = q_G_GU * r_GU * 3600 * 10 ** (-6)

    return E_G_GU


def calc_q_G_GU(q_out_hs, Theta_SW_hs):
    """ 1時間平均のガスユニットのガス消費量 (W) (14b)

    :param q_out_hs: 1時間平均の熱源機の暖房出力 (W)
    :type q_out_hs: ndarray
    :param Theta_SW_hs: 温水暖房用熱源機の往き温水温度
    :type Theta_SW_hs: ndarray
    :return: 1時間平均のガスユニットのガス消費量 (W)
    :rtype: ndarray
    """
    # ガスユニットの定格暖房効率
    e_GU_rtd = get_e_GU_rtd()

    # ガスユニットの定格暖房能力
    q_GU_rtd = get_q_GU_rtd()

    # ガスユニットの暖房能力
    q_GU = get_q_GU(q_out_hs, q_GU_rtd)

    return f_GU_G(q_GU, e_GU_rtd, q_GU_rtd, Theta_SW_hs)


def get_q_GU(q_out_hs, q_GU_rtd):
    """ ガスユニットの暖房能力 (14c)

    :param q_out_hs: 1時間平均の熱源機の暖房出力 (W)
    :type q_out_hs: ndarray
    :param q_GU_rtd: ガスユニットの定格暖房能力 (W)
    :type q_GU_rtd: float
    :return: ガスユニットの暖房能力 (14c)
    :rtype: ndarray
    """
    return np.clip(q_out_hs, None, q_GU_rtd)

ガスユニットの定格暖房効率$e_{rtd}$は0.87に等しいとし、ガスユニットの定格暖房能力$q_{GU,rtd}$は14000に等しいとする。 

The rated heating efficiency $ e_{rtd} $ of the gas unit is equal to 0.87, and the rated heating capacity $ q_{GU, rtd} $ of the gas unit is equal to 14000.

In [ ]:
def get_e_GU_rtd():
    """ ガスユニットの定格暖房効率 (-)

    :return: ガスユニットの定格暖房効率
    :rtype: float
    """
    return 0.87


def get_q_GU_rtd():
    """ ガスユニットの定格暖房能力 (W)

    :return: ガスユニットの定格暖房能力 (W)
    :rtype: float
    """
    return 14000

#### E.3.6 暖房出力分担率 / Heating output share rate

1 時間平均のヒートポンプユニットの暖房出力分担率$r_{HPU}$は0を下回る場合は0とし、1を上回る場合は1とする。 

The heating output allotment rate $ r_{HPU} $ of the 1-hour average heat pump unit is 0 if it is less than 0, and 1 if it is more than 1.

$$\large
r_{GU} = 1 - r_{HPU}
$$

$$\large
r_{HPU} = \frac{q_{HPU,max} - q'_{TU,tank,loss} - q'_{TU,pipe,loss}}{q_{out,hs} + q'_{GU,loss,HPU}}
$$

$q_{HPU,max}$：1 時間平均のヒートポンプユニットの最大暖房出力(W) / 1 hour average heat pump unit maximum heating output  
$q'_{TU,tank,loss}$：1 時間平均のヒートポンプユニット最大運転時のタンクユニットのタンクの熱損失(W) / Heat loss of the tank unit during maximum operation of the heat pump unit at one hour average  
$q'_{TU,pipe,loss}$：1 時間平均のヒートポンプユニット最大運転時のタンクユニットのヒートポンプ配管の熱損失(W) / Heat loss of heat pump piping of tank unit at maximum operation of heat pump unit at one hour average  
$q_{out,hs}$：1 時間平均の熱源機の暖房出力(W) / 1 hour average heating output of heat source machine  
$q'_{GU,loss,HPU}$：1 時間平均のヒートポンプユニット最大運転時のヒートポンプユニット運転時のガスユニットの熱損失(W) / Heat loss of gas unit during operation of heat pump unit at maximum operation of heat pump unit at one hour average  

In [ ]:
def get_r_GU(r_HPU):
    """ 1時間平均のガスユニットの暖房出力分担率 (15)

    :param r_HPU: 1時間平均のヒートポンプユニット暖房出力分担率 (-)
    :type r_HPU: ndarray
    :return: 1時間平均のガスユニットの暖房出力分担率
    :rtype: ndarray
    """
    return 1 - r_HPU


def get_r_HPU(q_HPU_max, q_dash_TU_tank_loss, q_dash_TU_pipe_loss, q_out_hs, q_dash_GU_loss_HPU):
    """ 1時間平均のヒートポンプユニットの暖房出力分担率 (16)

    :param q_HPU_max: 1時間平均のヒートポンプユニットの最大暖房出力 (W)
    :type q_HPU_max: ndarray
    :param q_dash_TU_tank_loss: 1時間平均のヒートポンプユニット最大運転時のタンクユニットのタンクの熱損失 (W)
    :type q_dash_TU_tank_loss: ndarray
    :param q_dash_TU_pipe_loss: 1時間平均のヒートポンプユニット最大運転時のタンクユニットのヒートポンプ配置の熱損失 (W)
    :type q_dash_TU_pipe_loss: ndarray
    :param q_out_hs: 1時間平均の熱源機の暖房出力 (W)
    :type q_out_hs: ndarray
    :param q_dash_GU_loss_HPU: 1時間平均のヒートポンプユニット最大運転時のヒートポンプユニット運転時のガスユニットの熱損失 (W)
    :type q_dash_GU_loss_HPU: ndarray
    :return: 1時間平均のヒートポンプユニットの暖房出力分担率 (16)
    :rtype: ndarray
    """
    r_HPU = np.zeros(24*365)
    f = (q_out_hs + q_dash_GU_loss_HPU > 0)
    r_HPU[f] = (q_HPU_max[f] - q_dash_TU_tank_loss[f] - q_dash_TU_pipe_loss[f]) / (q_out_hs[f] + q_dash_GU_loss_HPU[f])
    return np.clip(r_HPU, 0, 1)

$$\large
q_{HPU,max} = f_{HPU,q,max} (\theta_{ex}, \theta_{RW,hs}, \theta'_{HPU,out})
$$

$\theta_{ex}$：1 時間平均の外気温度(℃) / 1 hour average outside temperature  
$\theta_{RW,hs}$：1 時間平均の熱源機の戻り温水温度(℃) / One hour average return hot water temperature of heat source machine  
$\theta'_{HPU,out}$：1 時間平均のヒートポンプ最大運転時のヒートポンプユニットの出湯温度(℃) / Hot water temperature of heat pump unit at maximum heat pump operation at one hour average

In [ ]:
def calc_q_HPU_max(Theta_ex, Theta_RW_hs):
    """ 1時間平均のヒートポンプユニットの最大暖房出力 (17)

    :param Theta_ex: 1時間平均の外気温度 (℃)
    :type Theta_ex: ndarray
    :param Theta_RW_hs: 1時間平均の熱源機の戻り温水温度 (℃)
    :type Theta_RW_hs: ndarray
    :return: ヒートポンプユニットの最大暖房出力 (W)
    :rtype: ndarray
    """
    # 1時間平均のヒートポンプ大運転時のヒートポンプユニットの出湯温度
    Theta_dash_HPU_out = get_Theta_dash_HPU_out()

    # ヒートポンプユニットの最大暖房出力を求める関数 (26)
    q_HPU_max = f_HPU_q_max(Theta_ex, Theta_RW_hs, Theta_dash_HPU_out)

    return q_HPU_max

$$\large
q'_{TU,pipe,loss} = f_{TU,q,HPP} (\theta_{SW,hs}, \theta_{ex})
$$

$\theta_{SW,hs}$：1 時間平均の熱源機の往き温水温度(℃) / 1 hour average hot water temperature of the heat source machine  
$\theta_{ex}$：1 時間平均の外気温度(℃) / 1 hour average outside temperature  

In [ ]:
def calc_q_dash_TU_pipe_loss(Theta_SW_hs, Theta_ex):
    """ 1時間平均のヒートポンプ最大運転時のタンクユニットのヒートポンプ配管の熱損失 (18)

    :param Theta_SW_hs: 温水暖房用熱源機の往き温水温度
    :type Theta_SW_hs: ndarray
    :param Theta_ex: 外気温度
    :type Theta_ex: ndarray
    :return: 1時間平均のヒートポンプ最大運転時のタンクユニットのヒートポンプ配管の熱損失
    :rtype: ndarray
    """
    # タンクユニットのヒートポンプ配管の熱損失を求める関数 (28)
    q_dash_TU_pipe_loss = f_TU_q_HPP(Theta_SW_hs, Theta_ex)

    return q_dash_TU_pipe_loss

$$\large
q'_{TU,tank,loss} = f_{TU,q,tank} (\theta'_{HPU,out}, \theta_{TU,amb})
$$

$\theta'_{HPU,out}$：1 時間平均のヒートポンプ最大運転時のヒートポンプユニットの出湯温度(℃) / Hot water temperature of heat pump unit at maximum heat pump operation at one hour average  
$\theta_{TU,amb}$：1 時間平均のタンクユニットの周囲空気温度(℃) / 1 hour average tank unit ambient air temperature  

In [ ]:
def calc_q_dash_TU_tank_loss(Theta_TU_amb):
    """ ヒートポンプ最大運転時のタンクユニットのタンクの熱損失 (19)

    :param Theta_TU_amb: 1時間平均のタンクユニットの周囲空気温度 (℃)
    :type Theta_TU_amb: ndarray
    :return: ヒートポンプ最大運転時のタンクユニットのタンクの熱損失 (19)
    :rtype: ndarray
    """
    # 1時間平均のヒートポンプ最大運転時のヒートポンプユニットの出湯温度
    Theta_dash_HPU_out = get_Theta_dash_HPU_out()

    # タンクユニットのタンクの熱損失を求める関数 (29)
    q_dash_TU_tank_loss = f_TU_q_tank(Theta_dash_HPU_out, Theta_TU_amb)

    return q_dash_TU_tank_loss

$$\large
q'_{GU,loss,HPU} = f_{GU,q,lossHP} (\theta_{SW,hs}, \theta_{TU,amb})
$$

$\theta_{SW,hs}$：1 時間平均の熱源機の往き温水温度(℃) / 1 hour average hot water temperature of the heat source machine  
$\theta_{TU,amb}$：1 時間平均のタンクユニットの周囲空気温度(℃) / 1 hour average tank unit ambient air temperature  

In [ ]:
def calc_q_dash_GU_loss_HPU(Theta_SW_hs, Theta_TU_amb):
    """ ヒートポンプ最大運転時のヒートポンプユニット運転時のガスユニットの熱損失 (20)

    :param Theta_SW_hs: 1時間平均の熱源機の往き温水温度 (℃)
    :type Theta_SW_hs: ndarray
    :param Theta_TU_amb: 1時間平均のタンクユニットの周囲空気温度 (℃)
    :type Theta_TU_amb: ndarray
    :return: ヒートポンプ最大運転時のヒートポンプユニット運転時のガスユニットの熱損失
    :rtype: ndarray
    """
    # ヒートポンプユニット運転時のガスユニットの熱損失を求める関数 (32)
    q_dash_GU_loss_HPU = f_GU_q_lossHP(Theta_SW_hs, Theta_TU_amb)

    return q_dash_GU_loss_HPU

1 時間平均のヒートポンプ最大運転時のヒートポンプユニットの出湯温度$\theta'_{HPU,out}$ は 85℃する。 

Heat pump unit hot water temperature $ \theta'_{HPU, out} $ at 1 hour average heat pump maximum operation is 85 ° C.

In [ ]:
def get_Theta_dash_HPU_out():
    """ 1時間平均のヒートポンプ最大運転時のヒートポンプユニットの出湯温度 (℃)

    :return: 1時間平均のヒートポンプ最大運転時のヒートポンプユニットの出湯温度 (℃)
    :rtype: float
    """
    return 85.0


#### E.3.7 熱源機の平均暖房出力及び熱源機最大暖房出力 / Average heating output of heat source machine and maximum heating output of heat source machine

$$\large
q_{out,hs} = Q_{out,hs} \times 10^{6} \div 3600
$$

$Q_{out,hs}$：1時間当たりの熱源機の暖房出力(MJ/h) / Heating output of heat source unit per hour  

In [ ]:
def get_q_out_hs(Q_out_hs):
    """ 1時間平均の熱源機の暖房出力 (21)

    :param Q_out_hs: 1時間当たりの熱源機の暖房出力 (MJ/h)
    :type Q_out_hs: ndarray
    :return: 1時間平均の熱源機の暖房出力 (W)
    :rtype: ndarray
    """
    return Q_out_hs * 10 ** 6 / 3600

1 時間当たりの熱源機の暖房出力$Q_{out,hs}$は、本文により定義される、1 時間当たりの熱源機の最大暖房出力$Q_{max,hs}$を超えない範囲で 1 時間当たりの温水熱需要$Q_{dmd,hs}$に等しい。 

Heating output $ Q_{out, hs} $ of heat source machine per hour is defined by the text, within 1 hour not exceeding maximum heating output $ Q_{max, hs} $ of heat source machine per hour Equal to the per-minute hot water heat demand $ Q_{dmd, hs} $.

$$\large
Q_{max,hs} = q_{GU,rtd} \times 3600 \times 10^{-6}
$$

$q_{GU,rtd}$：ガスユニットの定格暖房能力(W) / Gas unit rated heating capacity  

In [ ]:
def get_Q_max_hs(q_GU_rtd):
    """ 1時間当たりの熱源機の最大暖房出力 (22)

    :param q_GU_rtd: ガスユニットの定格暖房能力 (W)
    :type q_GU_rtd: float
    :return: 1時間当たりの熱源機の最大暖房出力 (MJ/h)
    :rtype: float
    """
    Q_max_hs = q_GU_rtd * 3600 * 10 ** (-6)
    return Q_max_hs


#### E.3.8 タンクユニット周囲の空気温度 / Air temperature around tank unit

タンクユニットを室内に設置する場合

When installing the tank unit indoors

$$\large
\theta_{TU,amb} = 0.25 \times \theta_{ex} + 0.75 \times \theta_{in} 
$$

タンクユニットを室外に設置する場合

When installing the tank unit outdoors

$$\large
\theta_{TU,amb} = \theta_{ex}
$$

$\theta_{ex}$：1 時間平均の外気温度(℃) / 1 hour average outside temperature  
$\theta_{in}$：1 時間平均の室内温度(℃) / 1 hour average indoor temperature  

In [ ]:
def calc_Theta_TU_amb(Theta_ex, TU_place):
    """ 1時間平均のタンクユニットの周囲空気温度 (23)

    :param Theta_ex: 1時間平均の外気温度 (℃)
    :type Theta_ex: ndarray
    :param TU_place: タンクユニットの設置場所
    :type TU_place: str
    :return: 1時間平均のタンクユニットの周囲空気温度 (℃)
    :rtype: float
    """
    if TU_place == '屋内':
        # 1時間平均の室内温度
        Theta_in = get_Theta_in()
        return 0.25 * Theta_ex + 0.75 * Theta_in  # (23a)
    elif TU_place == '屋外':
        return Theta_ex  # (23b)
    else:
        raise ValueError(TU_place)

暖房運転時において、1 時間平均の室内温度$\theta_{in}$は、20℃とする。

The room temperature $ \theta_{in} $, which is an hourly average, is 20°C during heating operation.

In [ ]:
def get_Theta_in():
    """ 暖房時運転における1時間平均の室内温度 (℃)

    :return: 暖房時運転における1時間平均の室内温度 (℃)
    :rtype: float
    """
    return 20.0


### E.4 評価関数 / Evaluation function

#### E.4.1 ヒートポンプユニット / Heat pump unit

$$
f_{HPU,P} (\theta_{ex}, h_{ex},\theta_{RW,hs},\theta_{HP,out}, q_{HPU,out}) = (0.1869 \times P^{2} + 0.2963 \times P + 0.565) \times 10^{3}
$$

$$
P =  0.42108 \times q_{HPU,out} \times 10^{-3} - 0.03889 \times \theta_{ex} - 0.00762 \times h_{ex} +  0.03313 \times \theta_{RW,hs}\\ + 0.00449 \times \theta_{HP,out} - 0.82066
$$

$\theta_{ex}$：外気温度(℃) / Outside air temperature  
$h_{ex}$：外気相対湿度(％) / Relative humidity  
$q_{HPU,out}$：ヒートポンプユニットの平均暖房出力(W) / Average heating output of heat pump unit  
$\theta_{RW,hs}$：熱源機の戻り温水温度(℃) / Heat return temperature of heat source machine  
$\theta_{HP,out}$：ヒートポンプユニットの出湯温度(℃) / Hot water temperature of heat pump unit  

In [ ]:
def f_HPU_p(Theta_ex, h_ex, Theta_RW_hs, Theta_HP_out, q_HPU_out):
    """ ヒートポンプユニットの消費電力を求める関数 (24)

    :param Theta_ex: 外気温度
    :type Theta_ex: ndarray
    :param h_ex: 外気相対湿度
    :type h_ex: ndarray
    :param Theta_RW_hs: 1時間平均の熱源機の戻り温水温度 (℃)
    :type Theta_RW_hs: ndarray
    :param Theta_HP_out: ヒートポンプユニットの出湯温度 (℃)
    :type Theta_HP_out: ndarray
    :param q_HPU_out: ヒートポンプユニットの平均暖房出力 (W)
    :type q_HPU_out: ndarray
    :return: ヒートポンプユニットの消費電力を求める関数 (24)
    :rtype: ndarray
    """
    P = 0.42108 * q_HPU_out * 10 ** (-3) \
        - 0.03889 * Theta_ex \
        - 0.00762 * h_ex \
        + 0.03313 * Theta_RW_hs \
        + 0.00449 * Theta_HP_out \
        - 0.82066
    return (0.1869 * P ** 2 + 0.2963 * P + 0.565) * 10 ** 3

ヒートポンプユニットの最小暖房出力

Minimum heating output of heat pump unit

$$
f_{HPU,q,min} (\theta_{ex}, \theta_{RW,hs},\theta_{HPU,out}) = 6.08 \times \theta_{ex} - 46.56 \times \theta_{RW,hs} +0.04 \times \theta_{HPU,out} + 4739.62
$$

In [ ]:
def f_HPU_q_min(Theta_ex, Theta_RW_hs, Theta_HPU_out):
    """ ヒートポンプユニットの最小暖房出力を求める関数 (25)

    :param Theta_ex: 外気温度
    :type Theta_ex: ndarray
    :param Theta_RW_hs: 1時間平均の熱源機の戻り温水温度 (℃)
    :type Theta_RW_hs: ndarray
    :param Theta_HPU_out: 1時間平均のヒートポンプユニットの出湯温度 (℃)
    :type Theta_HPU_out: ndarray
    :return: ヒートポンプユニットの最小暖房出力を求める関数 (25)
    :rtype: ndarray
    """
    return 6.08 * Theta_ex - 46.56 * Theta_RW_hs + 0.04 * Theta_HPU_out + 4739.62

ヒートポンプユニットの最大暖房出力

Maximum heating output of heat pump unit

$$
f_{HPU,q,max} (\theta_{ex}, \theta_{RW,hs},\theta_{HPU,out}) = 1.36 \times \theta_{ex} - 89.85 \times \theta_{RW,hs} +  46.5 \times \theta_{HPU,out} +  5627.07
$$

$\theta_{ex}$：外気温度(℃) / Outside air temperature  
$\theta_{RW,hs}$：熱源機の戻り温水温度(℃) / Heat return temperature of heat source machine  
$\theta_{HPU,out}$：ヒートポンプユニットの出湯温度(℃) / Hot water temperature of heat pump unit  

In [ ]:
def f_HPU_q_max(Theta_ex, Theta_RW_hs, Theta_HPU_out):
    """ ヒートポンプユニットの最大暖房出力を求める関数 (26)

    :param Theta_ex: 外気温度
    :type Theta_ex: ndarray
    :param Theta_RW_hs: 1時間平均の熱源機の戻り温水温度 (℃)
    :type Theta_RW_hs: ndarray
    :param Theta_HPU_out: 1時間平均のヒートポンプユニットの出湯温度 (℃)
    :type Theta_HPU_out: ndarray
    :return: ヒートポンプユニットの最大暖房出力を求める関数 (26)
    :rtype: ndarray
    """
    return 1.36 * Theta_ex - 89.85 * Theta_RW_hs + 46.5 * Theta_HPU_out + 5627.07

#### E.4.2 タンクユニット / Tank unit

$$\large
f_{TU,P,aux} (r_{intmit}) = 5 + 10 + r_{intmit}
$$

$r_{intmit}$：ヒートポンプユニットの断続運転率 / Intermittent operation rate of heat pump unit 

In [ ]:
def f_TU_P_aux(r_intmit):
    """ タンクユニットの補機の消費電力を求める関数 (27)

    :param r_intmit: 1時間平均のヒートポンプユニットの断続運転率
    :type r_intmit: ndarray
    :return: タンクユニットの補機の消費電力 (W)
    :rtype: ndarray
    """
    return 5 + 10 * r_intmit

$$\large
f_{TU,q,HPP} (\theta_{SW,hs}, \theta_{ex}) = 2.8 \times (\theta_{SW,hs} - \theta_{ex})
$$

$\theta_{SW,hs}$：熱源機の往き温水温度(℃) / Hot water temperature of the heat source machine  
$\theta_{ex}$：外気温度(℃) / Outside air temperature  

In [ ]:
def f_TU_q_HPP(Theta_SW_hs, Theta_ex):
    """ タンクユニットのヒートポンプ配管の熱損失を求める関数 (28)

    :param Theta_SW_hs: 熱源機の往き温水温度 (℃)
    :type Theta_SW_hs: ndarray
    :param Theta_ex: 外気温度 (℃)
    :type Theta_ex: ndarray
    :return: タンクユニットのヒートポンプ配管の熱損失 (W)
    :rtype: ndarray
    """
    return 2.8 * (Theta_SW_hs - Theta_ex)

$$\large
f_{TU,q,tank} (\theta_{HP,out}, \theta_{TU,amb}) = max(0, 1.6 \times (\theta_{HP,out} - \theta_{TU,amb})-72.1)
$$

$\theta_{HP,out}$：ヒートポンプユニットの出湯温度(℃) / Hot water temperature of heat pump unit  
$\theta_{TU,amb}$：タンクユニットの周囲空気温度(℃) / Ambient temperature of tank unit  

In [ ]:
def f_TU_q_tank(Theta_HP_out, Theta_TU_amb):
    """ タンクユニットのタンクの熱損失を求める関数 (29)

    :param Theta_HP_out: ヒートポンプユニットの出湯温度 (℃)
    :type Theta_HP_out: float
    :param Theta_TU_amb: タンクユニットの周囲空気温度 (℃)
    :type Theta_TU_amb: ndarray
    :return: タンクユニットのタンクの熱損失 (W)
    :rtype: ndarray
    """
    return np.clip(1.6 * (Theta_HP_out - Theta_TU_amb) - 72.1, 0, None)

#### E.4.3 ガスユニット / Gas unit

$$\large
f_{GU,G} (q_{GU},e_{GU,rtd},q_{GU,rtd}) = (q_{GU} + q_{body}) \div e_{ex}
$$

$$\large
e_{ex} = e_{GU,rtd} \times f_{ex} \times (q_{GU,rtd} + q_{body}) \div q_{GU,rtd}
$$

$q_{GU}$：ガスユニットの暖房出力(W) / Gas unit heating output  
$q_{body}$：ガスユニットの筐体放熱損失(W) / Heat dissipation loss of gas unit case  
$e_{ex}$：ガスユニット燃焼室の熱交換効率 / Heat exchange efficiency of gas unit combustion chamber  
$f_{ex}$：ガスユニットの定格暖房効率を補正する係数 / Correction factor for rated heating efficiency of gas unit  
$e_{GU,rtd}$：ガスユニットの定格暖房効率 / Rated heating efficiency of gas unit  
$q_{GU,rtd}$：ガスユニットの定格暖房能力(W) / Gas unit rated heating capacity  

In [ ]:
def f_GU_G(q_GU, e_GU_rtd, q_GU_rtd, Theta_SW_hs):
    """ ガスユニットのガス消費量を求める関数 (30,31)

    :param q_GU: ガスユニットの暖房出力 (W)
    :type q_GU: ndarray
    :param e_GU_rtd: ガスユニットの定格暖房能力 (W)
    :type e_GU_rtd: float
    :param q_GU_rtd: ガスユニットの定格暖房能力 (W)
    :type q_GU_rtd: float
    :param Theta_SW_hs: 熱源機の往き温水温度 (℃)
    :type Theta_SW_hs: ndarray
    :return: ガスユニットのガス消費量を求める関数 (30,31)
    :rtype: ndarray
    """
    # f_ex, q_body の作業領域確保
    f_ex = np.zeros(24 * 365)
    q_body = np.zeros(24 * 365)

    # Theta_SW_hs == 40 の場合
    f1 = Theta_SW_hs == 40
    f_ex[f1] = get_table_e_3()[0][0]
    q_body[f1] = get_table_e_3()[1][0]

    # Theta_SW_hs == 60 の場合
    f2 = Theta_SW_hs == 60
    f_ex[f2] = get_table_e_3()[0][1]
    q_body[f2] = get_table_e_3()[1][1]

    e_ex = e_GU_rtd * f_ex * (q_GU_rtd + q_body) / q_GU_rtd

    return (q_GU + q_body) / e_ex

<div style="text-align: center;font-weight: bold;">表　ガスユニットの定格暖房効率を補正する係数及びガスユニットの筐体放熱損失<br>Table: Factors to correct the rated heating efficiency of the gas unit and the casing heat dissipation loss of the gas unit</div>

<table>
<thead>
<tr>
<th rowspan="2"></th>
<th colspan="2">熱源機の往き温水温度 $\theta_{SW,hs}$<br>Hot water temperature $\theta_{SW,hs}$ of the heat source machine</th>
</tr>
<tr>
<th>40℃の場合<br>At 40°C</th>
<th>60℃の場合<br>At 60°C</th>
</tr>
</thead>

<tbody>
<tr>
<th>ガスユニットの定格暖房効率を補正する係数 $f_{ex}$<br>Correction factor $f_{ex}$ for rated heating efficiency of gas unit</th>
<td>1.064</td>
<td>1.038</td>
</tr>
<tr>
<th>ガスユニットの筐体放熱損失 $q_{body}$<br>Heat dissipation loss $q_{body}$ of gas unit case</th>
<td>123.74</td>
<td>225.26</td>
</tr>

</tbody>
</table>


In [ ]:
def get_table_e_3():
    """ 表E.3 ガスユニットの定格暖房効率を補正する係数及びガスユニットの筐体放熱損失

    :return: 表E.3 ガスユニットの定格暖房効率を補正する係数及びガスユニットの筐体放熱損失
    :rtype: list
    """
    table_e_3 = [
        (1.064, 1.038),
        (123.74, 225.26)
    ]
    return table_e_3


$$\large
f_{GU,q,lossHP} (\theta_{SW,hs}, \theta_{TU,amb}) = max(5 \times (\theta_{SW,hs} - \theta_{TU,amb}) -100, 0)
$$

$\theta_{SW,hs}$：熱源機の往き温水温度(℃) / Hot water temperature of the heat source machine  
$\theta_{TU,amb}$：タンクユニットの周囲空気温度(℃) / Ambient temperature of tank unit  

In [ ]:
def f_GU_q_lossHP(Theta_SW_hs, Theta_TU_amb):
    """ ヒートポンプユニット運転時のガスユニットの熱損失を求める関数 (32)

    :param Theta_SW_hs: 温水暖房用熱源機の往き温水温度
    :type Theta_SW_hs: ndarray
    :param Theta_TU_amb: 1時間平均のタンクユニットの周囲空気温度 (℃)
    :type Theta_TU_amb: ndarray
    :return: ヒートポンプユニット運転時のガスユニットの熱損失を求める関数
    :rtype: ndarray
    """
    return np.clip(5 * (Theta_SW_hs - Theta_TU_amb) - 100, 0, None)

$$\large
f_{GU,P,aux} (q_{G,GU}) =73 + 0.003 \times q_{G,GU}
$$

$q_{G,GU}$：ガスユニットのガス消費量(W) / Gas unit gas consumption  

In [ ]:
def f_GU_P_aux(q_G_GU):
    """ ガスユニットの補機の消費電力を求める関数 (33)

    :param q_G_GU: ガスユニットのガス消費量 (W)
    :type q_G_GU: ndarray
    :return: ガスユニットの補機の消費電力 (W)
    :rtype: ndarray
    """
    return 73 + 0.003 * q_G_GU

### E.5 電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機（給湯熱源：ガス瞬間式、暖房熱源：電気ヒートポンプ・ガス瞬間式併用）の定義 / Definition of electric heat pump, gas moment type combined hot water supply hot water heater (hot water supply heat source: gas moment type, heating heat source: electric heat pump, gas moment type combined)

省略

ommision

### E.6 品番の追加（参考） / Add part number (reference)

省略

ommision